In [1]:
from ModelClass import RegressionModel
from DatasetClass import Dataset, DatasetMass
import matplotlib.pyplot as plt
import numpy as np
import itertools
from src.helpers import make_filter_slice
import tensorflow as tf
import optuna 
from optuna_dashboard import run_server
import threading
import sys
import logging 


2025-03-19 19:56:27.725808: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-03-19 19:56:27.736873: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1742410587.748965 3539757 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1742410587.752677 3539757 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-19 19:56:27.766440: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [2]:
erik_data = "/scratch/ucjf-atlas/htautau/SM_Htautau_R22/V02_skim_mva_01/*/*/*/*/*H125*.root"
patrik_data = "/scratch/ucjf-atlas/htautau/SM_Htautau_R22/V02_skim_mva_01/*/*/*/*/*Ztt*.root"


In [3]:
dataset = DatasetMass(file_paths=patrik_data, file_name = "data")
dataset.load_data()
print(dataset.train_events)
    
#dataset.augment_data_phi(n_slices=1000)

#train_batches = dataset.train_dataset.batch(32)
#for feature, target in train_batches.take(1):
#    print(f"Feature: {feature.shape}")
#    print(f"Target: {target.shape}")

#print(len(dataset.train_dataset))
#print(len(dataset.dev_dataset))
#print(len(dataset.val_dataset))
#This block of code iterates through the dataset and extracts the pt values of the labels and stores them in a list
#data = [labels.numpy()[0] for features, labels in dataset.train_dataset.take(100000)]

#plt.hist(data, bins=100, range=(50, 130), histtype='step', label='pt distribution', density=False)
#plt.legend(loc='upper right')
#plt.title('pt distribution of the dataset')
#plt.xlabel('pt')
#plt.ylabel('Number of events')
#plt.show()



I0000 00:00:1742410596.261099 3539757 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 43493 MB memory:  -> device: 0, name: NVIDIA L40S, pci bus id: 0000:41:00.0, compute capability: 8.9


7799005


In [14]:
dataset.make_slices(n_slices=100)
slices = dataset.slices   

lorentz_mask = tf.constant(dataset.get_lorentz_mask())  # Shape [35] bool values
lorentz_indices = tf.squeeze(tf.where(lorentz_mask), axis=1) # [0 1 2 3 4 5 6 7 13 14 ...] 
n_vectors = tf.shape(lorentz_indices)[0] // 4 # number of 4-vectors
lorentz_indices_4d = tf.reshape(lorentz_indices, (n_vectors, 4))  # [n_vectors, 4]

@tf.function
def augment_lorentz(data, target):
    beta = tf.random.uniform(shape = (), minval=-0.98, maxval=0.98) # Shape ()
    #tf.print("used beta: ", beta)
    gamma = 1.0 / tf.sqrt(1.0 - beta**2) # Shape ()
    #tf.print("used gamma: ", gamma)
    
    for i in range(n_vectors):
        vec_indices = lorentz_indices_4d[i] # Shape (4,)
        pt = data[vec_indices[0]] # scalar values
        eta = data[vec_indices[1]]
        #tf.print("eta: ", eta)
        phi = data[vec_indices[2]]
        E = data[vec_indices[3]]
        #tf.print("E: ", E)
        #print(E.shape)
        
        # Convert to Cartesian coordinates
        #px = pt * tf.cos(phi)
        #py = pt * tf.sin(phi)
        pz = pt * tf.sinh(eta) # Shape ()
        #tf.print("pz: ", pz)

        E_prime = gamma * (E - beta * pz) # Shape ()
        #tf.print("E_prime: ", E_prime)
        pz_prime = gamma * (pz - beta * E) 
        
        epsilon = 1e-8
        eta_prime = tf.asinh(pz_prime / (pt + epsilon)) # Shape ()
        #tf.print("eta_prime: ", eta_prime)
        update_indices = tf.reshape(vec_indices, [-1, 1])  # Shape (4, 1)
        update_values = tf.stack([pt, eta_prime, phi, E_prime]) # Shape (4,)
        # Update the tensor
        data = tf.tensor_scatter_nd_update(
            data,
            indices=update_indices,
            updates=update_values
        )
        #print(data.shape)
        
    return data, target

n_events = 10000
new_dataset = tf.data.Dataset.sample_from_datasets([s.repeat() for s in slices], weights=[1.]*len(slices))
#orig_features, orig_masses = next(iter(new_dataset))
#print(orig_features)
#print(orig_masses)
new_dataset = new_dataset.take(n_events)
augmented_dataset = new_dataset.map(augment_lorentz)
#new_features, new_masses = next(iter(new_dataset))
#print(new_features)š
#print(new_masses)
#batch_dataset = augmented_dataset.batch(n_events)
#print(f"Number of events in batch_dataset: {len(augmented_dataset)}")
#features, masses = next(iter(batch_dataset))

#plt.hist(masses, range=(70, 130), bins=50)
#plt.show()

                                                                    

In [ ]:

lorentz_mask = tf.constant(dataset.get_lorentz_mask()) 
print(lorentz_mask)

features = tf.constant([
    5.56329613e+01, -3.16469967e-01, -9.03481603e-01, 6.74349565e-07,
    5.47039528e+01, -1.49002433e-01, 2.52283901e-01, 1.05658375e-01,
    1.67467535e-01, 1.15576553e+00, 1.16783524e+00, 1.38526611e+02,
    1.10336914e+02, 1.38102692e+02, 1.74350947e-01, 2.87319326e+00,
    1.55200872e+01, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
    0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
    0.00000000e+00, 4.71827660e+01, 0.00000000e+00, -6.81111366e-02,
    -1.58858541e-02, 1.00000000e+00, 1.00000000e+00, 1.00000000e+00,
    0.00000000e+00, 1.00000000e+00, 1.00000000e+00
], dtype=tf.float32)

label = tf.constant(94.92591, dtype=tf.float32)

dataset = tf.data.Dataset.from_tensors((features, label))
dataset = dataset.map(augment_lorentz)

new_features, new_label = next(iter(dataset))
print(new_features)
print(new_label)

tf.Tensor(
[ True  True  True  True  True  True  True  True False False False False
 False  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True False False False False False False], shape=(35,), dtype=bool)
tf.Tensor(
[ 5.56329613e+01 -3.16706300e-01 -9.03481603e-01 -7.03347385e-01
  5.47039528e+01 -1.49041474e-01  2.52283901e-01 -2.15698794e-01
  1.67467535e-01  1.15576553e+00  1.16783524e+00  1.38526611e+02
  1.10336914e+02  1.38102692e+02  1.78831533e-01  2.87319326e+00
  1.64828949e+01  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  4.71827660e+01 -1.32284049e-05 -6.81111366e-02
 -1.58981103e-02  1.00000000e+00  1.00000000e+00  1.00000000e+00
  0.00000000e+00  1.00000000e+00  1.00000000e+00], shape=(35,), dtype=float32)
tf.Tensor(94.92591, shape=(), dtype=float32)


In [15]:
dataset.make_slices(n_slices=10)
slices = dataset.slices 
phi_mask = tf.constant(dataset.get_phi_mask())

@tf.function
def augment_phi(data, target):
    # generate random rotation angle
    angle = tf.random.uniform(shape=(tf.shape(data)[0],), minval=-np.pi, maxval=np.pi)
    #tf.print("angle: ", angle.shape)
    #tf.print("data: ", data.shape)
    # apply rotation
    data  = tf.where(phi_mask, data + angle, data)
    
    # normalize angles between -pi and pi
    data = tf.where(phi_mask, tf.math.atan2(tf.sin(data), tf.cos(data)), data)
    
    return data, target

# sample from the slices
n_events = 1000
new_dataset = tf.data.Dataset.sample_from_datasets([s.repeat() for s in slices], weights=[1.]*len(slices))
new_dataset = new_dataset.take(n_events)

# apply augmentation
new_dataset = new_dataset.map(augment_phi)
#aug_dataset = new_dataset.batch(n_events)


#print(f"Number of events in batch_dataset: {len(batch_dataset)}")
#features, masses = next(iter(batch_dataset))
#print(features.shape)
#print(features)

#plt.hist(masses, range=(70, 130), bins=50)
#plt.show()




In [ ]:
param_grid = {
    'batch_size': [3200, 6400],
    'learning_rate': [0.01, 0.1],
    'epochs': [5, 20, 30]
}

iterable = list(itertools.product(*param_grid.values()))
for params in iterable:
    model = RegressionModel(dataset=dataset, batch_size=params[0], initial_learning_rate=params[1], n_epochs=params[2])
    model.prepare_dataset()
    model.create_normalizer()
    model.build_model()
    model.train_model()
    model.plot_history()

Batching datasets...
Building model...
Training model...
Epoch 1/5


2025-03-17 20:42:51.802874: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


305/305 ━━━━━━━━━━━━━━━━━━━━ 48s 151ms/step - loss: 2335.7473 - mean_squared_error: 2335.7437 - val_loss: 312.1779 - val_mean_squared_error: 312.4890
Epoch 2/5
305/305 ━━━━━━━━━━━━━━━━━━━━ 46s 150ms/step - loss: 251.6451 - mean_squared_error: 251.6452 - val_loss: 255.4529 - val_mean_squared_error: 255.5452
Epoch 3/5
305/305 ━━━━━━━━━━━━━━━━━━━━ 45s 147ms/step - loss: 221.5373 - mean_squared_error: 221.5372 - val_loss: 222.5968 - val_mean_squared_error: 222.6384
Epoch 4/5
305/305 ━━━━━━━━━━━━━━━━━━━━ 45s 149ms/step - loss: 208.6559 - mean_squared_error: 208.6557 - val_loss: 204.9115 - val_mean_squared_error: 204.9481
Epoch 5/5
305/305 ━━━━━━━━━━━━━━━━━━━━ 46s 152ms/step - loss: 201.6959 - mean_squared_error: 201.6958 - val_loss: 201.3717 - val_mean_squared_error: 201.4131


TypeError: RegressionModel.plot_history() takes 1 positional argument but 2 were given

In [ ]:
def objective(trial):

    model = RegressionModel(
                dataset,
                n_layers             = trial.suggest_int('n_layers', 1, 5),
                hidden_layer_size    = trial.suggest_int('hidden_layer_size', 32, 1024),
                initial_learning_rate= trial.suggest_float('initial_learning_rate', 1e-4, 1e-2, log=True),
                n_epochs             = trial.suggest_int('n_epochs', 10, 100, step=10),
                activation_function  = 'relu',
                batch_size           = trial.suggest_int('batch_size', 512, 6556,step=512),

    )

    model.prepare_dataset()
    model.create_normalizer()
    model.build_model()
    model.train_model()

    val_loss, _ = model.model.evaluate(model.val_batch, verbose=0)
    return val_loss

study = optuna.create_study(storage="sqlite:///optuna.db", study_name="Higgs_analysis_mass_2", direction='minimize', load_if_exists=True)

def run_dashboard():
    server = run_server("sqlite:///optuna.db", host="0.0.0.0", port=8080)
    server.run()  # Run dashboard continuously in a separate thread

dashboard_thread = threading.Thread(target=run_dashboard)
dashboard_thread.daemon = True  # Ensures it stops when the script ends
dashboard_thread.start()

def run_optuna():
    study.optimize(objective, n_trials=100, n_jobs=1)

# Run Optuna in a separate thread so the dashboard can be accessed in real-time
optuna_thread = threading.Thread(target=run_optuna)
optuna_thread.start()

optuna_thread.join()  # Wait for Optuna to finish before printing results
print("Number of finished trials:", len(study.trials))
print("Best trial:", study.best_trial.params)



[I 2025-03-19 19:57:35,300] A new study created in RDB with name: Higgs_analysis_mass_2
Bottle v0.13.2 server starting up (using WSGIRefServer())...
Listening on http://0.0.0.0:8080/
Hit Ctrl-C to quit.

/singularity/ucjf/venv_tf_218/lib64/python3.9/site-packages/optuna/distributions.py:708: UserWarning: The distribution is specified by [512, 6556] and step=512, but the range is not divisible by `step`. It will be replaced by [512, 6144].
  warnings.warn(


Batching datasets...


2025-03-19 19:57:35.801286: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


Building model...
Training model...
Epoch 1/80


I0000 00:00:1742410656.815979 3539925 service.cc:148] XLA service 0x7fcda400a810 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1742410656.816007 3539925 service.cc:156]   StreamExecutor device (0): NVIDIA L40S, Compute Capability 8.9
2025-03-19 19:57:36.835403: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1742410656.944428 3539925 cuda_dnn.cc:529] Loaded cuDNN version 90300
2025-03-19 19:57:37.571764: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:397] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_226', 100 bytes spill stores, 100 bytes spill loads

2025-03-19 19:57:37.576017: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:397] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_226', 24 bytes spill store

...

173/174 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step - loss: 4259.8467 - mean_squared_error: 4259.8467

2025-03-19 19:57:50.514674: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:397] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_226', 56 bytes spill stores, 44 bytes spill loads

2025-03-19 19:57:50.647109: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:397] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_226', 372 bytes spill stores, 372 bytes spill loads

2025-03-19 19:57:50.651348: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:397] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_226', 772 bytes spill stores, 772 bytes spill loads

2025-03-19 19:57:50.791720: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:397] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_226', 1084 bytes spill stores, 1084 bytes spill loads

2025-03-19 19:57:50.899901: I external/local_xla/xla/str

174/174 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step - loss: 4245.9995 - mean_squared_error: 4245.9668

2025-03-19 19:58:15.853295: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:397] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_45', 56 bytes spill stores, 44 bytes spill loads

2025-03-19 19:58:15.948622: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:397] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_45_0', 200 bytes spill stores, 200 bytes spill loads

2025-03-19 19:58:15.956436: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:397] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_45', 76 bytes spill stores, 76 bytes spill loads

2025-03-19 19:58:15.968422: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:397] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_45', 372 bytes spill stores, 372 bytes spill loads

2025-03-19 19:58:16.013585: I external/local_xla/xla/stream_ex

174/174 ━━━━━━━━━━━━━━━━━━━━ 40s 216ms/step - loss: 4232.3101 - mean_squared_error: 4232.2451 - val_loss: 418.3317 - val_mean_squared_error: 419.2327
Epoch 2/80
174/174 ━━━━━━━━━━━━━━━━━━━━ 33s 189ms/step - loss: 315.6752 - mean_squared_error: 315.6960 - val_loss: 269.3182 - val_mean_squared_error: 269.9005
Epoch 3/80
174/174 ━━━━━━━━━━━━━━━━━━━━ 34s 195ms/step - loss: 248.7186 - mean_squared_error: 248.7384 - val_loss: 250.1487 - val_mean_squared_error: 250.6241
Epoch 4/80
174/174 ━━━━━━━━━━━━━━━━━━━━ 34s 197ms/step - loss: 232.7969 - mean_squared_error: 232.8143 - val_loss: 250.3391 - val_mean_squared_error: 250.6977
Epoch 5/80
174/174 ━━━━━━━━━━━━━━━━━━━━ 34s 194ms/step - loss: 226.2121 - mean_squared_error: 226.2267 - val_loss: 258.8209 - val_mean_squared_error: 259.0879
Epoch 6/80
173/174 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step - loss: 223.4304 - mean_squared_error: 223.4304